# Sri Lankan Census Data Visualisation

### Let's first import the libraries we need

In [ ]:
# import matplotlib package for plotting 
import matplotlib
matplotlib.use('TkAgg') # this is a backend that lets us use interactive plotting

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable 

We will be using a Sri Lankan Geodata library to generate Sri Lankan maps.
See more at https://medium.com/on-technology/plotting-maps-of-sri-lanka-with-geopandas-and-matplotlib-377d50e3f0f

In [ ]:
# Sri Lankan geodata library 
from geo import geodata 

In [ ]:
# pandas is the industry standard python library for tabular data
import pandas as pd 

In [ ]:
# inline makes plots within the notebook
%matplotlib inline

## Part 1: Do we need more teachers in Sri Lankan schools?

We are using data downloaded from various SL government sources. Most data is available in PDF form only. This is bad as we can't perform analysis on pdf files. There are Python tools that let you convert pdfs to csv format. One such tool is Tabula. You can read more about it here: https://tabula.technology/ <br>

For this class, I have already created required csv files. But when you convert from pdfs, most will have some errors that need fixing. We will get a flavour for data cleaning in the class but often data can be more messy than here. 

### 1. Get district wise student and teacher stats

In [ ]:
# pandas produces objects called DataFrames 
district_school_stats = pd.read_csv('tabula-School_Census_Report_2019.csv')

In [ ]:
# see the structure of data
district_school_stats.head(10)

In [ ]:
district_school_s = district_school_stats.rename(columns={
    'Unnamed: 0' : 'district', 
    'Unnamed: 1' : 'total number of schools',
    'Unnamed: 2' : 'percentage_schools',
    'Unnamed: 3' : 'total number of students', 
    'Unnamed: 4': 'percentage_students',
    'Total No.': 'total number of teachers', 
    'Unnamed: 6': 'percentage_teachers', 
    'Student': 'student teacher ratio',
    'Average' : 'average number of students per school',
    'Average' : 'average number of teachers per school'})

In [ ]:
district_school_s = district_school_s.loc[4:,:]

In [ ]:
district_school_s

#### Remove numbers from district names

In [ ]:
district_school_s['district'] = (district_school_s.iloc[:-1]['district']).apply(lambda x: x.split('.')[1].strip())

#### Clean up student numbers column

In [ ]:
district_school_s['total number of students']= district_school_s.iloc[:-1]['total number of students'].apply(lambda x: (x.replace(',', '')))

In [ ]:
district_school_s['total number of students']= district_school_s.iloc[:-1]['total number of students'].apply(lambda x: (x.replace(' ', '')))

In [ ]:
district_school_s['total number of students']= district_school_s.iloc[:-1]['total number of students'].apply(lambda x: float(x))

#### Clean up teacher numbers column

In [ ]:
district_school_s['total number of teachers']= district_school_s.iloc[:-1]['total number of teachers'].apply(lambda x: (x.replace(',', '')))

In [ ]:
district_school_s['total number of teachers']= district_school_s.iloc[:-1]['total number of teachers'].apply(lambda x: (x.replace(' ', '')))

In [ ]:
district_school_s['total number of teachers']= district_school_s.iloc[:-1]['total number of teachers'].apply(lambda x: float(x))

In [ ]:
district_school_s = district_school_s[district_school_s['district'].notnull()]

In [ ]:
district_school_s.iloc[:, 1:] = district_school_s.iloc[:, 1:].astype('float64')

### 2. Get district wise geodata

In [ ]:
district_geo = geodata.get_region_geodata('LK', 'district')

In [ ]:
district_geo = district_geo.rename(columns={'name':'district'})

### 3. Merge district school stats and district geodata into one DataFrame

In [ ]:
district_stats = pd.merge(district_geo, district_school_s, on='district', how='inner')

In [ ]:
district_stats['teachers per 100 students'] = (1/district_stats['student teacher ratio'])*100

In [ ]:
district_stats['teachers per 100 students'] = (district_stats['total number of teachers']/district_stats['total number of students'])*100

### 4. Plot teacher to student ratio on Sri Lanka map

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
ax.set_aspect('equal')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=1)
district_stats.plot(column='teachers per 100 students', ax=ax, cax=cax, legend=True, cmap=  'OrRd')

ax.set_title('District Teachers to Student Ratio', fontsize=16)
cax.set_ylabel('Teachers per 100 students', rotation=90, fontsize=14)
plt.tight_layout()

plt.savefig('teacher_student_ration.png')

### 5. Get O/L pass rate

In [ ]:
pass_rate = pd.read_csv('tabula-o-l-pass-rate.csv')

#### Clean up ol_pass rate dataframe

In [ ]:
pass_rate = pass_rate.set_index('district')

In [ ]:
pass_rate = pass_rate.rename(index={'Monaragala':'Moneragala'})

In [ ]:
pass_rate = pass_rate.reset_index()

### 6. Merge O/L passrate DataFrame with district geo DataFrame

In [ ]:
pass_rate_geo = pd.merge(district_geo, pass_rate, on='district', how='inner')

In [ ]:
pass_rate_geo['percentage qualified for a_l'] = pass_rate_geo['percentage qualified for a_l'].astype('float64')

### 7. Plot O/L pass rate into one and teacher student ratio into another side by side

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15,12))
axs[0].set_aspect('equal')
axs[1].set_aspect('equal')
divider_0 = make_axes_locatable(axs[0])
divider_1 = make_axes_locatable(axs[1])

cax_0 = divider_0.append_axes("right", size="5%", pad=.5)
cax_1 = divider_1.append_axes("right", size="5%", pad=.5)


district_stats.plot(column='teachers per 100 students', ax=axs[0], cax=cax_0, legend=True, cmap='OrRd')

axs[0].set_title('District teacher student ratio', fontsize=18)
cax_0.set_ylabel("teachers per 100 students", rotation=90, fontsize=15)

pass_rate_geo.plot(column='percentage qualified for a_l', ax=axs[1], cax=cax_1, legend=True, cmap='OrRd')

axs[1].set_title('O/L pass rate for 2018', fontsize=18)
cax_1.set_ylabel('percentage qualified for A/L from 2018 O/L examination', rotation=90, fontsize=15)


axs[0].tick_params(which='both', left=False, bottom=False, labelleft=False, labelbottom=False)
axs[1].tick_params(which='both', left=False, bottom=False, labelleft=False, labelbottom=False)

plt.tight_layout()
# plt.savefig('teachers_vs_o_l_pass_rate.jpg')

### 8. Scatter plot teacher student ration vs. O/L pass rate and find best fit line

In [ ]:
dist_pass_merge = pd.merge(district_stats, pass_rate, on='district', how='inner')

In [ ]:
import sklearn.linear_model
import numpy as np

X = np.c_[dist_pass_merge["teachers per 100 students"]]
y = np.c_[dist_pass_merge["percentage qualified for a_l"]]


# Select a linear model
model = sklearn.linear_model.LinearRegression()

# Train the model
model.fit(X, y)

# Make a prediction for teacher ratio 10
X_new = [[10]]  # high teacher ratio
print(model.predict(X_new)) # outputs 

In [ ]:
from sklearn import linear_model # scikit learn is a machine learning library

dist_pass_merge.plot(kind='scatter', x='teachers per 100 students', y='percentage qualified for a_l', figsize=(10,8))
lin1 = linear_model.LinearRegression()
x, y = np.c_[dist_pass_merge['teachers per 100 stu dents']], np.c_[dist_pass_merge['percentage qualified for a_l']]

lin1.fit(x, y)
t0, t1 = lin1.intercept_[0], lin1.coef_[0][0]
t0, t1
X=np.linspace(4, 9, 1000)
plt.plot(X, t0 + t1 * X, 'r')

### 9. Are A/L outcomes any different?

In [ ]:
al_result = pd.read_csv('al-2020-results.csv')

In [ ]:
al_result['percentage'] = al_result['number_uni_eligible'].apply(lambda x: x.split()[1])

In [ ]:
al_result['number_uni_eligible']  = al_result['number_uni_eligible'].apply(lambda x: x.split()[0])

In [ ]:
al_result = al_result[['district', 'number_sat', 'number_uni_eligible', 'percentage']]

In [ ]:
al_result

In [ ]:
grad_teachers = pd.read_csv('teachers_by_category.csv')

In [ ]:
grad_teachers['District'] = grad_teachers['District'][:-1].apply(lambda x: x.split('.')[1].strip())

In [ ]:
grad_teachers.loc[25, 'District'] = 'Total'

In [ ]:
grad_teachers = grad_teachers.rename(columns={'District': 'district'})

In [ ]:
al_outcomes = pd.merge(al_result, grad_teachers, on='district', how='inner')

In [ ]:
al_outcomes['number_sat'] = al_outcomes['number_sat'].apply(lambda x: x.replace(',', ''))

In [ ]:
al_outcomes['percentage'] = al_outcomes['percentage'].astype('float64')

In [ ]:
al_outcomes

In [ ]:
al_outcomes['grad teachers per 100 a_l students'] = (al_outcomes['Graduate']/al_outcomes['number_sat'])*100

In [ ]:
al_outcomes.plot(kind='scatter', x='grad teachers per 100 a_l students', y='percentage', figsize=(10,8))

lin1 = linear_model.LinearRegression()
x, y = np.c_[al_outcomes['grad teachers per 100 a_l students']], np.c_[al_outcomes['percentage']]

lin1.fit(x, y)
t0, t1 = lin1.intercept_[0], lin1.coef_[0][0]
t0, t1
X = np.linspace(34, 70, 10)
plt.plot(X, t0 + t1 * X, 'r')


plt.ylabel('percentage eligible for university')
plt.xlabel('number of graduate teachers per 100 A/L exam sitters (new syllabus) in 2020')

In [ ]:
t0

In [ ]:
t1

## Part 2: Will the people of Mannar survive a Tsunami? 

In [ ]:
mannar_main = geodata.get_region_geodata('LK-42', 'gnd')

In [ ]:
mannar_main['gnd_centroid'] = mannar_main['geometry'].centroid

In [ ]:
mannar = mannar_main
mannar = mannar.sort_values(by=['centroid_altitude']).reset_index()

In [ ]:
lowest_altitude = mannar.iloc[:3,:]

In [ ]:
lowest_altitude

In [ ]:
highest_altitude = mannar.iloc[-3:,:].reset_index()

In [ ]:
highest_altitude

In [ ]:
mannar['population_density'] = mannar['population']/mannar['area']

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15,7))
axs[0].set_aspect('equal')
axs[1].set_aspect('equal')
divider_0 = make_axes_locatable(axs[0])
divider_1 = make_axes_locatable(axs[1])

cax_0 = divider_0.append_axes("right", size="5%", pad=.5)
cax_1 = divider_1.append_axes("right", size="5%", pad=.5)

mannar.plot(column='centroid_altitude', ax=axs[0], cax=cax_0, legend=True, cmap='OrRd')

axs[0].set_title('Mannar District Altitude by GN Division')
cax_0.set_ylabel('Altitude in meters', rotation=90)

mannar.plot(column='population_density', ax=axs[1], cax=cax_1, legend=True, cmap='OrRd', vmin=0, vmax=200)

axs[1].set_title('Mannar District Population Density by GN Division')
cax_1.set_ylabel("Population per square kilometer", rotation=90)

axs[0].tick_params(which='both', left=False, bottom=False, labelleft=False, labelbottom=False)
axs[1].tick_params(which='both', left=False, bottom=False, labelleft=False, labelbottom=False)

plt.tight_layout()
plt.savefig('mannar_population_density.jpg')

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))

ax.set_aspect('equal')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=1)
mannar.plot(column='centroid_altitude', ax=ax, cax=cax, legend=True, 
            cmap='OrRd')

ax.set_title('Mannar District GN Division Altitudes: Highest 3 and Lowest 3 ', fontsize=18)
cax.set_ylabel('Altitude in meters', rotation=90, fontsize=16)

ax.tick_params(which='both', left=False, bottom=False, labelleft=False, labelbottom=False)

highest_altitude['gnd_centroid'].plot(ax=ax, color='black', marker='o', markersize=5)
lowest_altitude['gnd_centroid'].plot(ax=ax, color='black', marker='o', markersize=5)

for i, name in enumerate(lowest_altitude['name']):
    ax.annotate(name, 
                xy=(lowest_altitude['gnd_centroid'][i].x, 
                    lowest_altitude['gnd_centroid'][i].y),
                xycoords='data',
                xytext=(0,5), textcoords='offset points', fontsize=14)


ax.annotate(highest_altitude['name'][0], 
            xy=(highest_altitude['gnd_centroid'][0].x, 
                highest_altitude['gnd_centroid'][0].y),
            xycoords='data',
            xytext=(0,10), 
            textcoords='offset points', 
            horizontalalignment='right',
            fontsize=14)

ax.annotate(highest_altitude['name'][1], 
            xy=(highest_altitude['gnd_centroid'][1].x, 
                highest_altitude['gnd_centroid'][1].y),
            xycoords='data',
            xytext=(-8,0), 
            textcoords='offset points', 
            horizontalalignment='right',
            fontsize=14)

ax.annotate(highest_altitude['name'][2], 
            xy=(highest_altitude['gnd_centroid'][2].x, 
                highest_altitude['gnd_centroid'][2].y),
            xycoords='data',
            xytext=(0,-35), 
            textcoords='offset points', 
            horizontalalignment='center',
            fontsize=14)

plt.tight_layout()
plt.savefig('mannar_altitude.jpg')

### Part 3: Patterns in data, a study of the population density distribution along Ampara's coast 

In [ ]:
east = geodata.get_region_geodata('LK-5', 'dsd')


In [ ]:
east_store = east.copy()

In [ ]:
east = east_store.copy()

In [ ]:
east = east[~east['district_id'].str.contains('LK-53')]

In [ ]:
east = east.reset_index()

In [ ]:
east['population density'] = east['population']/east['area']
east['dsd_centroid'] = east['geometry'].centroid

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
ax.set_aspect('equal')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=1)
east.plot(column='population density', ax=ax, cax=cax, legend=True, cmap=  'YlOrRd', vmin = 100, vmax=5000)


for i, name in enumerate(list(east['name'])):
    # print(i, name)
    ax.annotate(name, 
                xy=(east.loc[i,'dsd_centroid'].x, 
                    east.loc[i,'dsd_centroid'].y),
                xycoords='data',
                xytext=(0,5), textcoords='offset points', fontsize=8)

In [ ]:
east = east[east['dsd_centroid'].x > 81.4 ]

In [ ]:
list(east['name'])

In [ ]:
east = east[~east['name'].str.contains('Lahugala|Damana|Ampara|Uhana')]

In [ ]:
east = east[~east['name'].str.contains('Koralai Pattu Central|Oddamavadi|Valaichchenai|Eravur Pattu|West|')]

In [ ]:
east = east[~east['name'].str.contains('Eravur|Valachchenai|Porativu|Thirukkovil|Pothuvil')]

In [ ]:
east = east.drop('index', axis=1)

In [ ]:
east.reset_index(inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
ax.set_aspect('equal')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=1)
east.plot(column='population density', ax=ax, cax=cax, legend=True,cmap=  'OrRd', vmin = 700, vmax=4000)

east['dsd_centroid'].plot(ax=ax, color='black', marker='o', markersize=5)
annotate_kalmunai['centroid'].plot(ax=ax, color='black', marker='o', markersize=5)


for i, name in enumerate(list(east['name'])):
    # print(i, name)
    ax.annotate(name, 
                xy=(east.loc[i,'dsd_centroid'].x, 
                    east.loc[i,'dsd_centroid'].y),
                xycoords='data',
                xytext=(0,5), textcoords='offset points', fontsize=12)


for i, name in enumerate(list(annotate_kalmunai['name'])):
        ax.annotate(name, 
                xy=(annotate_kalmunai.loc[i,'centroid'].x, 
                    annotate_kalmunai.loc[i,'centroid'].y),
                xycoords='data',
                xytext=(0,5), textcoords='offset points', fontsize=12)
,

In [ ]:
annotate_kalmunai = kalmunai_gnd[kalmunai_gnd.index % 9 == 0]

In [ ]:
annotate_kalmunai = annotate_kalmunai[annotate_kalmunai['gnd_id'].str.contains('LK-5221015|LK-5224040|LK-5224130')]

In [ ]:
annotate_kalmunai = annotate_kalmunai.reset_index()

In [ ]:
annotate_kalmunai.loc[0,'name'] = 'Kalmunaikudi'
annotate_kalmunai.loc[1, 'name'] = 'Periyaneelavanai'
annotate_kalmunai.loc[2, 'name'] = 'Maruthamunai'

In [ ]:
annotate_kalmunai

In [ ]:
east = east[east['name'].str.contains('Manmunai South & Eruvil pattu|Ninthavur|Karaitheevu|Sainthamaruthu|Kalmunai')]

In [ ]:
east = east.drop('index', axis=1)

In [ ]:
east = east.reset_index()

In [ ]:
ampara = geodata.get_region_geodata('LK-52', 'dsd')


In [ ]:
ampara = ampara.reset_index()

In [ ]:
ampara['gnd_centroid'] = ampara['geometry'].centroid

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
ax.set_aspect('equal')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=1)
ampara.plot(column='population', ax=ax, cax=cax, legend=True, cmap=  'YlOrRd', vmin = 500, vmax=100000)

for i, name in enumerate(list(ampara['name'])):
    # print(i, name)
    ax.annotate(name, 
                xy=(ampara.loc[i,'gnd_centroid'].x, 
                    ampara.loc[i,'gnd_centroid'].y),
                xycoords='data',
                xytext=(0,5), textcoords='offset points', fontsize=8)

In [ ]:
ampara['name']

In [ ]:
lk_gnd =  geodata.get_region_geodata(''Manmunai South & Eruvil pattu''Manmunai South & Eruvil pattu'LK', 'gnd')

In [ ]:
lk_gnd.head()

In [ ]:
lk_dsd = geodata.get_region_geodata('LK', 'dsd')

In [ ]:
kalmunai = lk_dsd[lk_dsd['name'].str.contains('Ninthavur')]

In [ ]:
kalmunai

In [ ]:
kalmunai_gnd = lk_gnd[lk_gnd['dsd_id'].str.contains('LK-5221|LK-5224|LK-5227|LK-5225')].copy()

In [ ]:
kalmunai_gnd['centroid'] = kalmunai_gnd['geometry'].centroid
kalmunai_gnd['density'] = kalmunai_gnd['population']/kalmunai_gnd['area']

In [ ]:
kalmunai_gnd = kalmunai_gnd.reset_index()

In [ ]:
kalmunai_gnd = kalmunai_gnd.drop('index', axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
ax.set_aspect('equal')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=1)

dots = kalmunai_gnd[kalmunai_gnd.index % 7 == 0]

kalmunai_gnd.plot(column='density', ax=ax, cax=cax, cmap='OrRd', legend=True, vmin=500, vmax=20000)

dots['centroid'].plot(ax=ax, color='black', marker='o', markersize=5)

for i, name in enumerate(list(kalmunai_gnd['name'])):
        if i % 7 !=0:
            continue
        ax.annotate(name, 
                xy=(kalmunai_gnd.loc[i,'centroid'].x, 
                    kalmunai_gnd.loc[i,'centroid'].y),
                xycoords='data',
                xytext=(0,5), textcoords='offset points', fontsize=14)


In [ ]:
kalmunai_gnd['density'].min()

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
east.plot(ax=ax, edgecolor='black')

In [ ]:
ls